In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install SimpleITK
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import SimpleITK as sitk
from google.colab.patches import cv2_imshow
import os
from PIL import Image
from math import *
from random import *
from sklearn.decomposition import PCA

In [ ]:
import os
import SimpleITK as sitk
from skimage.color import rgb2hsv, hsv2rgb

adc_folder = '/content/drive/MyDrive/Dataset/crop_resize_ADC'
z_adc_folder = '/content/drive/MyDrive/Dataset/crop_resize_Z_ADC'
output_folder = '/content/drive/MyDrive/Fusion'

def process_image_pair(adc_image_path, z_adc_image_path, output_path):
    adc_image = sitk.ReadImage(adc_image_path)
    z_adc_image = sitk.ReadImage(z_adc_image_path)

    adc_data = sitk.GetArrayFromImage(adc_image)
    z_adc_data = sitk.GetArrayFromImage(z_adc_image)

    print(f"ADC shape: {adc_data.shape}, Z_ADC shape: {z_adc_data.shape}")

    if adc_data.shape != z_adc_data.shape:
        print(f"Skipping: Images have different shapes")
        return

   # Normalize the images
    adc_norm = adc_data / np.max(adc_data)
    z_adc_norm = z_adc_data / np.max(z_adc_data)

    # Convert to RGB
    adc_rgb = np.stack((adc_norm,)*3, axis=-1)
    z_adc_rgb = np.stack((z_adc_norm,)*3, axis=-1)

    # Convert to HSV
    adc_hsv = rgb2hsv(adc_rgb)
    z_adc_hsv = rgb2hsv(z_adc_rgb)

    # Fuse the images by taking the value from Z_ADC
    fused_hsv = np.copy(adc_hsv)
    fused_hsv[..., 2] = z_adc_hsv[..., 2]

    # Convert back to RGB
    fused_rgb = hsv2rgb(fused_hsv)

    # Convert to grayscale
    fused_gray = np.dot(fused_rgb[..., :3], [0.2989, 0.5870, 0.1140])

    # Convert the fused grayscale image back to SimpleITK format
    fused_image = sitk.GetImageFromArray(fused_gray)
    fused_image.CopyInformation(adc_image)  # Copy the metadata from ADC image

    sitk.WriteImage(fused_image, output_path)

    print(f"Fusion complete for images in {os.path.dirname(adc_image_path)}")


def create_z_adc_filename(adc_filename):
    base_name = adc_filename.replace('ADC_ss.mha', '')
    z_adc_filename = f"Zmap_{base_name}ADC_smooth2mm_clipped10.mha"
    return z_adc_filename

def process_folders(adc_folder, z_adc_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for adc_file in os.listdir(adc_folder):
        if adc_file.endswith('.mha'):
            print(f"Found ADC file: {adc_file}")
            adc_image_path = os.path.join(adc_folder, adc_file)
            z_adc_file = create_z_adc_filename(adc_file)
            z_adc_image_path = os.path.join(z_adc_folder, z_adc_file)

            if os.path.exists(z_adc_image_path):
                output_path = os.path.join(output_folder, f"fused_image_{adc_file}.mha")
                process_image_pair(adc_image_path, z_adc_image_path, output_path)
            else:
                print(f"Missing Z_ADC file: {z_adc_file}")

process_folders(adc_folder, z_adc_folder, output_folder)
print("All image fusions complete.")


In [ ]:
import glob
Fusion_paths = sorted(glob.glob('/content/drive/MyDrive/Fusion/*'))
Fusion = sitk.ReadImage(Fusion_paths)
Fusion = sitk.GetArrayFromImage(Fusion)
Fusion = np.moveaxis(Fusion, 1, -1)

In [ ]:
print(len(Fusion))
print(Fusion.shape)

In [ ]:
for i in range(Fusion.shape[3]):
    plt.imshow(Fusion[16, :, :, i], cmap='gray')
    plt.axis('off')
    plt.title(f'Slice {i}')
    plt.show()